## 03a - Assess Unique ILLs 
go through and get the number of outlinks between versions and compares their shared ILLs

import:

None
    
output:

    03a_onlyEnNoILL.txt - link English Wikipedia, doesn't have an Arabic ILL
    03a_enOutarILL.txt - in the english article and has an arabic ILL
    03a_onlyEn.txt - outlink only in the English article,  where the Arabic ILL is in the outlinks, this is arabic
    03a_onlyEn_english.txt - outlink only in the English article, where the Arabic ILL is in the outlinks, this is English

    03a_onlyAr.txt - in Arabic article where the English ILL is in the outlinks, this is English
    03a_onlyAr_arabic.txt - in Arabic article where the English ILL is in the outlinks, this is Arabic
    03a_onlyArNoILL.txt - link in Arabic Wikipedia, doesn't have an English ILL
    
    
    
The Plan:
- this goes through and does some initial work with the outlinks 
- it grabs all of the outlinks in the english and arabic versions of the arab spring
- organize them by ones that are only in each language. 
- identify ones that are in both by if there were english outlinks
- that had an arabic ILL and if that arabic ILL was also an outlink


1. Get Outlinks
2. Check ILL Symmetry - English
3. Check ILL Symmetry - Arabic


In [49]:

import numpy as np
import pandas as pd

import wikifunctions as wf #Brian's stuff

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

import json, requests

from urllib.parse import unquote


#----
import requests
from bs4 import BeautifulSoup

was having problems with wikifunctions so i just copied and pasted the code here and altered it for my code 

In [10]:
def parse_to_links(input,is_json=True):
    # Initialize an empty list to store the links
    outlinks_list = []
    
    if is_json:
        page_html = input['parse']['text']#['*']
    else:
        page_html = input

    # Parse the HTML into Beautiful Soup
    soup = BeautifulSoup(page_html,'lxml')

    # Remove sections at end
    bad_sections = ['See_also','Notes','References','Bibliography','External_links']
    bad_titles = ['Special:','Wikipedia:','Help:','Template:','Category:','International Standard','Portal:','s:','File:','Digital object identifier','(page does not exist)']
    
    sections = soup.find_all('h2')
    for section in sections:
#         print(section) #<h2 id="References">References</h2>
#         print(type(section)) #<class 'bs4.element.Tag'>
#         if section.span['id'] in bad_sections:#<------------------------------THIS IS THE PROBLEM
        if section['id'] in bad_sections:
            # Clean out the divs
            div_siblings = section.find_next_siblings('div')
            for sibling in div_siblings:
                sibling.clear()

            # Clean out the ULs
            ul_siblings = section.find_next_siblings('ul')
            for sibling in ul_siblings:
                sibling.clear()
                
    
    # Delete tags associated with templates
    for tag in soup.find_all('tr'):
        tag.replace_with('')

    # For each paragraph tag, extract the titles within the links
    for para in soup.find_all('p'):
        for link in para.find_all('a'):
            if link.has_attr('title'):
                title = link['title']
                # Ignore links that aren't interesting or are redlinks
                if all(bad not in title for bad in bad_titles) and 'redlink' not in link['href']:
                    outlinks_list.append(title)

    # For each unordered list, extract the titles within the child links
    for unordered_list in soup.find_all('ul'):
        for item in unordered_list.find_all('li'):
            for link in item.find_all('a'):
                if link.has_attr('title'):
                    title = link['title']
                    # Ignore links that aren't interesting or are redlinks
                    if all(bad not in title for bad in bad_titles) and 'redlink' not in link['href']:
                        outlinks_list.append(title)
    
    return outlinks_list

In [11]:
def get_page_outlinks(page_title, endpoint='en.wikipedia.org/w/api.php', redirects=1):
    """Takes a page title and returns a list of wiki-links on the page. The 
    list may contain duplicates and the position in the list is approximately 
    where the links occurred.
    
    page_title - a string with the title of the page on Wikipedia
    endpoint - a string that points to the web address of the API.
        This defaults to the English Wikipedia endpoint: 'en.wikipedia.org/w/api.php'
        Changing the two letter language code will return a different language edition
        The Wikia endpoints are slightly different, e.g. 'starwars.wikia.com/api.php'
    redirects - 1 or 0 for whether to follow page redirects, defaults to 1
    
    Returns:
    outlinks_per_lang - a dictionary keyed by language returning a dictionary 
        keyed by page title returning a list of outlinks
    """
    
    # Get the response from the API for a query
    # After passing a page title, the API returns the HTML markup of the current article version within a JSON payload
    #req = requests.get('https://{2}.wikipedia.org/w/api.php?action=parse&format=json&page={0}&redirects={1}&prop=text&disableeditsection=1&disabletoc=1'.format(page_title,redirects,lang))
    query_url = "https://{0}".format(endpoint)
    query_params = {}
    query_params['action'] = 'parse'
    query_params['page'] = page_title
    query_params['redirects'] = redirects
    query_params['prop'] = 'text'
    query_params['disableeditsection'] = 1
    query_params['disabletoc'] = 1
    query_params['format'] = 'json'
    query_params['formatversion'] = 2
    
    json_response = requests.get(url = query_url, params = query_params).json()
    
    if 'parse' in json_response.keys():
        links = parse_to_links(json_response)
        final_title = json_response['parse']['title']
    else:
        links = list()
        final_title = page_title
        
    return links 

## 1. Get Outlinks

In [51]:
#English
out_en_lyst = get_page_outlinks('Arab Spring')
print(len(out_en_lyst))

593


In [18]:
#Arabic
out_ar_lyst = get_page_outlinks('الربيع العربي', endpoint='ar.wikipedia.org/w/api.php')
print(len(out_ar_lyst))

493


### 1a. Resolve Redirects

In [19]:
out_en_lyst_noReDir = wf.resolve_redirects(out_en_lyst)
out_ar_lyst_noReDir = wf.resolve_redirects(out_ar_lyst,endpoint='ar.wikipedia.org/w/api.php')

print(len(out_en_lyst_noReDir))
print(len(out_ar_lyst_noReDir ))

517
382


### 1b. Delete Duplicates

In [20]:
#get rid of duplicates in English
out_en_lyst_clean = []

for x in out_en_lyst_noReDir:
    if x not in out_en_lyst_clean: 
        out_en_lyst_clean.append(x)



#the number of unique English outlinks
print(len(out_en_lyst_clean))

394


In [21]:
#get rid of duplicates Arabic
out_ar_lyst_clean = []

for x in out_ar_lyst_noReDir:
    if x not in out_ar_lyst_clean: 
        out_ar_lyst_clean.append(x)


#the number of unique Arabic outlinks
print(len(out_ar_lyst_clean))

275


## 2. Check ILL Symmetry - English

go through ILLs of outlinks and see if the ILL is in the outlink of the other language (english)

In [22]:
onlyEnNoILL = [] #a list of english links - this is an En ILL but no arabic ILL 
enOutarILL = [] #a list of english links - there is an arILL in out_ar_lyst_clean
onlyEn = [] #a list of english links- that have an arabic out link but it is not in in out_ar_lyst_clean


go through english outlinks get the ones that have arabic ILLs and where there are none

In [23]:
#gets all of the arabic ILL for english outlinks
#if there is one it is added to:
arILLforEnOutlinks = []

#if there isn't one it is added to: 
onlyEnNoILL = [] 

for enOut in out_en_lyst_clean:
    ILL = wf.get_interlanguage_links(enOut)
    try:
        arILL = ILL['ar']
        arILLforEnOutlinks.append(arILL)
        #print(arILL)
    except KeyError as ke:
        #print(enOut)
        onlyEnNoILL.append(enOut) #no arabic ILL 
    

In [24]:
#checks to see if the arabic ILLs are in the outlinks of the arabic page

#if they are add it to: 
enOutarILL = []
enOutarILL_arabic = []

#if the arabic ILL is not referenced also in the arabic page add it to
onlyEn = []
onlyEn_english = []

        
for arOut in arILLforEnOutlinks: #goes through all of the arabic outlinks
    ILL = wf.get_interlanguage_links(arOut, 'ar.wikipedia.org/w/api.php' )
    #print(ILL)
    enILL = ILL['en'] #goes backward to the english ILL
        
    if arOut in out_ar_lyst_clean: #there is arILL in the arabic outlinks 
        enOutarILL.append(enILL)
        enOutarILL_arabic.append(arOut)
        
    else:
        onlyEn.append(arOut) #this is if it has an arILl but it isn't in the outlinks
        onlyEn_english.append(enILL)
        

In [25]:
#Outlinks in English article that don't have an Arabic ILL
print(len(onlyEnNoILL))

#Outlinks in the English version of the article, and have an Arabic ILL in the Arabic version of the article
print(len(enOutarILL))
print(len(enOutarILL_arabic))

#Outlinks that have and Arabic ILL but it isn't in the Arabic version of the Arab Spring article
print(len(onlyEn)) 

31
82
82
281


In [26]:
len(out_en_lyst_clean)

394

Save Data

In [ ]:
with open("../Data/03a_onlyEnNoILL.txt", "w") as f:
    for s in onlyEnNoILL:
        f.write(str(s) +"\n")

In [ ]:
with open("../Data/03a_enOutarILL.txt", "w") as f:
    for s in enOutarILL:
        f.write(str(s) +"\n")

In [ ]:
with open("../Data/03a_onlyEn.txt", "w") as f:
    for s in onlyEn:
        f.write(str(s) +"\n")

In [ ]:
with open("../Data/03a_onlyEn_english.txt", "w") as f:
    for s in onlyEn_english:
        f.write(str(s) +"\n")

## 3. Check ILL Symmetry - Arabic

go through arabic outlinks

go through Arabic outlinks get the ones that have English ILLs and where there are none

In [31]:
#gets all of the english translations of the outlinks
#if there is an english ILL
enILLforArOutlinks = []
#if there isn't one it is added to: 
onlyArNoILL = [] 

for arOut in out_ar_lyst_clean: #goes throguh all the arabic outlines
    ILL = wf.get_interlanguage_links(arOut, endpoint='ar.wikipedia.org/w/api.php')
    try:
        enILL = ILL['en']
        enILLforArOutlinks.append(enILL) #gets english ILL
        #print(arILL)
    except KeyError as ke:
        onlyArNoILL.append(arOut) #no english ILL 
        

        

In [32]:
#checks to see if the engish ILLs are in the outlinks of the english page

#if they are add it to: 
arOutenILL = []
arOutenILL_english = []

#if the english ILL is not referenced also in the arabic page add it to
onlyAr = []
onlyAr_arabic = []
        
for enOut in enILLforArOutlinks: #goes through all the ones with ILLs in english
    ILL = wf.get_interlanguage_links(enOut, endpoint='ar.wikipedia.org/w/api.php')
    arILL = ILL['ar'] #translates them back to arabic
    #print(arILL)
        
    if enOut in out_en_lyst_clean: #there is enILL in the english outlinks 
        arOutenILL.append(arILL) #adds the arabic version
        arOutenILL_english.append(enOut) #keept the english version
        
    else:
        onlyAr.append(enOut) #this is if it has an arILl but it isn't in the outlinks
        onlyAr_arabic.append(arILL)

In [33]:
#Outlinks that are only in the Arabic Wikipedia and have no English ILL
print(len(onlyArNoILL))

# Outlinks that have an English ILL nad that English ILL is in the English version of the article
print(len(arOutenILL))
print(len(arOutenILL_english))

# OUtlinks that have and English ILL but it isn't mentioned in the English version of the article
print(len(onlyAr)) 

21
82
82
172


In [34]:
print(len(out_ar_lyst_clean))

275


Save Data

In [ ]:
with open("../Data/03a_onlyAr.txt", "w") as f:
    for s in onlyAr:
        f.write(str(s) +"\n")

In [ ]:
with open("../Data/03a_onlyAr_arabic.txt", "w") as f:
    for s in onlyAr_arabic:
        f.write(str(s) +"\n")

In [ ]:
with open("../Data/03a_onlyArNoILL.txt", "w") as f:
    for s in onlyArNoILL:
        f.write(str(s) +"\n")